Цель:
Применить на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения.

Описание задания:
В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)

In [ ]:
df = pd.read_csv('/content/heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


2. Подготовьте датасет к обучению моделей:
a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.


In [ ]:
categ_sign = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
df_dum = pd.get_dummies(df, columns=categ_sign, drop_first = True)
# df_dum.isna().sum().sum()
df_dum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                918 non-null    int64  
 1   RestingBP          918 non-null    int64  
 2   Cholesterol        918 non-null    int64  
 3   FastingBS          918 non-null    int64  
 4   MaxHR              918 non-null    int64  
 5   Oldpeak            918 non-null    float64
 6   HeartDisease       918 non-null    int64  
 7   Sex_M              918 non-null    uint8  
 8   ChestPainType_ATA  918 non-null    uint8  
 9   ChestPainType_NAP  918 non-null    uint8  
 10  ChestPainType_TA   918 non-null    uint8  
 11  RestingECG_Normal  918 non-null    uint8  
 12  RestingECG_ST      918 non-null    uint8  
 13  ExerciseAngina_Y   918 non-null    uint8  
 14  ST_Slope_Flat      918 non-null    uint8  
 15  ST_Slope_Up        918 non-null    uint8  
dtypes: float64(1), int64(6), u

3.Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_dum.drop(columns = ['HeartDisease']), df_dum['HeartDisease'], test_size=0.2, random_state=7)

4.Обучите модель логистической регрессии с параметрами по умолчанию.

In [ ]:
model_logistic = LogisticRegression(max_iter=1000)
# при дефолтном кол-ве итераций не всегда сходится =(
model_logistic.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
cross_validate(…, cv=10, scoring=['accuracy','recall','precision','f1'])

In [ ]:
score_0 = cross_validate(model_logistic,X_test, y_test, cv = 10, scoring = ['accuracy','recall','precision','f1'])

6. Оптимизируйте 3-4 параметра модели:
a) Используйте GridSearchCV.

In [ ]:
range_c = np.arange(0,3,0.1)
# penalty=['l1','l2','elasticnet']
param_grid = dict(penalty=['l1','l2','elasticnet'],solver=['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga'], C=range_c)
grid = GridSearchCV(model_logistic,param_grid, cv=10,scoring = 'accuracy',n_jobs=-1)
grid.fit(X_train,y_train)
# как же хорошо, что он не прекращает работы если параметры не могут быть использованы вместе <3
# просто пишет: 3360 fits failed out of a total of 5400.


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
3360 fits failed out of a total of 5400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
300 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueError: Sol

GridSearchCV(cv=10, estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'C': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
       2.6, 2.7, 2.8, 2.9]),
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='accuracy')

In [ ]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,param_solver,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001383,0.001125,0.0,0.0,0.0,l1,lbfgs,"{'C': 0.0, 'penalty': 'l1', 'solver': 'lbfgs'}",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238
1,0.005552,0.003022,0.0,0.0,0.0,l1,liblinear,"{'C': 0.0, 'penalty': 'l1', 'solver': 'libline...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,407
2,0.001626,0.002102,0.0,0.0,0.0,l1,newton-cg,"{'C': 0.0, 'penalty': 'l1', 'solver': 'newton-...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,406
3,0.000939,0.000067,0.0,0.0,0.0,l1,newton-cholesky,"{'C': 0.0, 'penalty': 'l1', 'solver': 'newton-...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,405
4,0.001952,0.002223,0.0,0.0,0.0,l1,sag,"{'C': 0.0, 'penalty': 'l1', 'solver': 'sag'}",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,0.000840,0.000077,0.0,0.0,2.9,elasticnet,liblinear,"{'C': 2.9000000000000004, 'penalty': 'elasticn...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,265
536,0.000834,0.000068,0.0,0.0,2.9,elasticnet,newton-cg,"{'C': 2.9000000000000004, 'penalty': 'elasticn...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264
537,0.000791,0.000059,0.0,0.0,2.9,elasticnet,newton-cholesky,"{'C': 2.9000000000000004, 'penalty': 'elasticn...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263
538,0.000788,0.000052,0.0,0.0,2.9,elasticnet,sag,"{'C': 2.9000000000000004, 'penalty': 'elasticn...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,447


In [ ]:
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

0.8664383561643836
{'C': 0.30000000000000004, 'penalty': 'l2', 'solver': 'newton-cg'}
LogisticRegression(C=0.30000000000000004, max_iter=1000, solver='newton-cg')


b) Используйте RandomizedSearchCV.

In [ ]:
grid_random = RandomizedSearchCV(estimator=model_logistic,param_distributions=param_grid,n_iter=300, cv=10,scoring = 'accuracy',n_jobs=-1)
grid_random.fit(X_train,y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
1920 fits failed out of a total of 3000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueError: Sol

RandomizedSearchCV(cv=10, estimator=LogisticRegression(max_iter=1000),
                   n_iter=300, n_jobs=-1,
                   param_distributions={'C': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
       2.6, 2.7, 2.8, 2.9]),
                                        'penalty': ['l1', 'l2', 'elasticnet'],
                                        'solver': ['lbfgs', 'liblinear',
                                                   'newton-cg',
                                                   'newton-cholesky', 'sag',
                                                   'saga']},
                   scoring='accuracy')

In [ ]:
print(grid_random.best_score_)
print(grid_random.best_params_)
print(grid_random.best_estimator_)
# запишу решатели в переменные
# С отличается на 0.1 для методов Grid_cv и random_cv
best_grid_cv = grid.best_estimator_
best_random_cv = grid_random.best_estimator_

0.8651055164753794
{'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.5}
LogisticRegression(C=0.5, max_iter=1000)


In [ ]:
print(grid.best_estimator_)

LogisticRegression(C=0.30000000000000004, max_iter=1000, solver='newton-cg')


In [ ]:
# решения друх методов  подставлю в модель для подсчета точности на тестовых данных
score_best_grid_cv = cross_validate(best_grid_cv,X_test, y_test, cv = 10, scoring = ['accuracy','recall','precision','f1'])
score_best_random_cv = cross_validate(best_random_cv,X_test, y_test, cv = 10, scoring = ['accuracy','recall','precision','f1'])


c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.

In [ ]:
# Для начала попробую модель слуяайного леса
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
param_grid = dict(n_estimators=list(range(10,170,25)), criterion=['gini','entropy','log_loss'], min_samples_split=list(range(0,15,2)))
grid_random_on_forest = RandomizedSearchCV(estimator=random_forest,param_distributions=param_grid,n_iter=300, cv=10,scoring = 'accuracy',n_jobs=-1)
grid_random_on_forest.fit(X_train,y_train)



/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 168 is smaller than n_iter=300. Running 168 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
700 fits failed out of a total of 1680.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
210 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py",

RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(), n_iter=300,
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'min_samples_split': [0, 2, 4, 6, 8, 10,
                                                              12, 14],
                                        'n_estimators': [10, 35, 60, 85, 110,
                                                         135, 160]},
                   scoring='accuracy')

In [ ]:
print(grid_random_on_forest.best_score_)
print(grid_random_on_forest.best_params_)
best_grid_random_on_forest = grid_random_on_forest.best_estimator_
print(grid_random_on_forest.best_estimator_)

0.8719548315438725
{'n_estimators': 35, 'min_samples_split': 14, 'criterion': 'entropy'}
RandomForestClassifier(criterion='entropy', min_samples_split=14,
                       n_estimators=35)


In [ ]:
# раз метрика не меняется, попробую улучшить на худшем дереве (в дз_ансамблирование)
# до этого же уровня, что б убедиться что этот способ работает)
# на DecisionTreeClassifier accuracy = 0.79
from sklearn.tree import DecisionTreeClassifier
tree_decision = DecisionTreeClassifier()
param_grid = dict(criterion=['gini','entropy','log_loss'], splitter=['best','random', ], max_depth = list(range(2,100,5)))

grid_random_on_decision_tree = RandomizedSearchCV(estimator=tree_decision,param_distributions=param_grid,n_iter=300, cv=10,scoring = 'accuracy',n_jobs=-1)
grid_random_on_decision_tree.fit(X_train,y_train)



/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 120 is smaller than n_iter=300. Running 120 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
400 fits failed out of a total of 1200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
400 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/tree/_classes.py", li

RandomizedSearchCV(cv=10, estimator=DecisionTreeClassifier(), n_iter=300,
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [2, 7, 12, 17, 22, 27, 32,
                                                      37, 42, 47, 52, 57, 62,
                                                      67, 72, 77, 82, 87, 92,
                                                      97],
                                        'splitter': ['best', 'random']},
                   scoring='accuracy')

In [ ]:
print(grid_random_on_decision_tree.best_score_)
print(grid_random_on_decision_tree.best_params_)
grid_random_on_decision_tree = grid_random_on_forest.best_estimator_
print(grid_random_on_decision_tree)
# сработало=) подняла точность на 10 % от старого дз

0.8433728248796741
{'splitter': 'random', 'max_depth': 7, 'criterion': 'gini'}
RandomForestClassifier(criterion='entropy', min_samples_split=14,
                       n_estimators=35)


d) Повторите п. 5 после каждого итогового изменения параметров.

In [ ]:
acc = {}
# заполню средними по всем метрикам
acc['0'] = [score_0[i].mean() for i in score_0.keys()]
score_best_grid_cv = cross_validate(best_grid_cv,X_test, y_test, cv = 10, scoring = ['accuracy','recall','precision','f1'])

acc['score_best_grid_cv'] = [score_best_grid_cv[i].mean() for i in score_best_grid_cv.keys()]
score_best_random_cv = cross_validate(best_random_cv,X_test, y_test, cv = 10, scoring = ['accuracy','recall','precision','f1'])

acc['score_best_random_cv'] = [score_best_random_cv[i].mean() for i in score_best_random_cv.keys()]

score_best_random_forest = cross_validate(best_grid_random_on_forest,X_test, y_test, cv = 10, scoring = ['accuracy','recall','precision','f1'])

acc['best_random_forest'] = [score_best_random_forest[i].mean() for i in score_best_random_forest.keys()]

score_best_random_decision_tree = cross_validate(grid_random_on_decision_tree,X_test, y_test, cv = 10, scoring = ['accuracy','recall','precision','f1'])
acc['best_random_decision_tree'] = [score_best_random_decision_tree[i].mean() for i in score_best_random_decision_tree.keys()]

df_res = pd.DataFrame.from_dict(acc, orient = 'index',columns = score_0.keys())
df_res.sort_values(['test_accuracy'])
# score_best_random_cv незначительно выигрывает по доли правильных ответов алгоритма

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
best_random_forest,0.049122,0.008030,0.847076,0.857778,0.843283,0.845556
score_best_random_cv,0.121306,0.004799,0.857895,0.857778,0.865628,0.856426
best_random_decision_tree,0.048944,0.007643,0.863158,0.878889,0.855429,0.864918
score_best_grid_cv,0.024706,0.004816,0.868713,0.890000,0.859874,0.871901
0,0.218207,0.006796,0.869006,0.868889,0.873405,0.866402


In [ ]:
df_res.sort_values(['fit_time'])
# best_random_forest незначительно выигрывает по времени обучения 

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
score_best_grid_cv,0.024706,0.004816,0.868713,0.890000,0.859874,0.871901
best_random_decision_tree,0.048944,0.007643,0.863158,0.878889,0.855429,0.864918
best_random_forest,0.049122,0.008030,0.847076,0.857778,0.843283,0.845556
score_best_random_cv,0.121306,0.004799,0.857895,0.857778,0.865628,0.856426
0,0.218207,0.006796,0.869006,0.868889,0.873405,0.866402


In [ ]:
# тот же результат, соответсвенно, по  оценки по времени
df_res.sort_values(['score_time'])

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
score_best_random_cv,0.121306,0.004799,0.857895,0.857778,0.865628,0.856426
score_best_grid_cv,0.024706,0.004816,0.868713,0.890000,0.859874,0.871901
0,0.218207,0.006796,0.869006,0.868889,0.873405,0.866402
best_random_decision_tree,0.048944,0.007643,0.863158,0.878889,0.855429,0.864918
best_random_forest,0.049122,0.008030,0.847076,0.857778,0.843283,0.845556


In [ ]:
# score_best_grid_cv слегка опережает по полноте положительного класса
df_res.sort_values(['test_recall'])


,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
score_best_random_cv,0.121306,0.004799,0.857895,0.857778,0.865628,0.856426
best_random_forest,0.049122,0.008030,0.847076,0.857778,0.843283,0.845556
0,0.218207,0.006796,0.869006,0.868889,0.873405,0.866402
best_random_decision_tree,0.048944,0.007643,0.863158,0.878889,0.855429,0.864918
score_best_grid_cv,0.024706,0.004816,0.868713,0.890000,0.859874,0.871901


In [ ]:
df_res.sort_values(['test_precision'])
# дефолтная логистическая регрессия выигрывает по угадыванию положительных объектов

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
best_random_forest,0.049122,0.008030,0.847076,0.857778,0.843283,0.845556
best_random_decision_tree,0.048944,0.007643,0.863158,0.878889,0.855429,0.864918
score_best_grid_cv,0.024706,0.004816,0.868713,0.890000,0.859874,0.871901
score_best_random_cv,0.121306,0.004799,0.857895,0.857778,0.865628,0.856426
0,0.218207,0.006796,0.869006,0.868889,0.873405,0.866402


In [ ]:
# по f-мере score_best_grid_cv - золотой чемпион
# однако и тут дефолтная отстаёт не сильно
df_res.sort_values(['test_f1'])


,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
best_random_forest,0.049122,0.008030,0.847076,0.857778,0.843283,0.845556
score_best_random_cv,0.121306,0.004799,0.857895,0.857778,0.865628,0.856426
best_random_decision_tree,0.048944,0.007643,0.863158,0.878889,0.855429,0.864918
0,0.218207,0.006796,0.869006,0.868889,0.873405,0.866402
score_best_grid_cv,0.024706,0.004816,0.868713,0.890000,0.859874,0.871901


7. Сформулируйте выводы по проделанной работе:
a) Сравните метрики построенных моделей.
b) *Сравните с полученными результатами в домашнем задании по теме «Ансамблирование». dz_ensemble


по-моему не стоит мешать прекрасным алгоритмам логистической регрессии, со встроенными параметрами
Они так же хорошо.

можно лишь отметить небольшое превосходство по времени у best_random_forest, однако тот же рандом при логистической был помедленннее. 
Вообщем я бы в этой задаче не стала применять подбор параметров. Как говорится: "можно, но зачем =)"

По сравнению с дефолтным ансамблированием, в самом плохом случае у DecisionTreeClassifier, точность test_accuracy подрасла почти на 8%, это уже маленькая, но победа. 



